In [615]:
# import libraries
# DataFrame
import pandas as pd

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# sklearny
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB


#tensorflow
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

# Utility
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
import string
import pickle
# RNN - LSTM
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import re
import keras
from keras import regularizers
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Test Input dataset before model training

In [616]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))
print(stopword)

{'all', 'needn', 'because', 'up', "she'll", 'or', "you've", "shan't", 'over', 'where', 'wasn', 'into', 'out', "i'll", 'that', 'yourself', 'by', 'same', 'at', 'before', 'hasn', 'll', 'mustn', 'don', 'when', 'it', 'hadn', 'very', "they'd", 'you', 'after', 'the', "you'd", 'their', 'those', "mightn't", 'which', 'only', 'what', 'i', 'has', 'will', 'can', 'each', 'she', 're', "that'll", "they've", 'no', 'this', "hasn't", "he'd", 'down', "needn't", 'isn', 'm', 'her', 'between', 'whom', 'such', 'is', 'just', 'hers', "hadn't", 'haven', "i'm", 'and', 'most', 'own', "they're", "i've", 'there', 'during', 'through', "he's", 'ours', 'doing', 'am', 'mightn', "it'll", 'we', 'how', 'itself', 'y', 'our', 'any', 'once', 'few', 'wouldn', 'are', 'd', 'these', 'as', 'ma', 'himself', 'both', "couldn't", 'having', 'me', 'than', "it's", 'o', "you'll", 'be', "doesn't", 'not', 'some', "we're", "won't", 't', 's', 'had', 'against', "i'd", 'aren', 'should', 'nor', 'have', 'further', 'with', 'a', 'too', "aren't", 't

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhaijingjing/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [617]:
negation_words = {"no", "nor", "not", "don", "don't", "didn", "didn't", "doesn", "doesn't", "can't", "cannot", "won't", "wouldn", "wouldn't", "shouldn", "shouldn't", "isn't", "aren't", "wasn", "wasn't", "weren't", "hadn", "hadn't", "", "wouldn't"}
custom_stop_words = stopword.difference(negation_words)
print(custom_stop_words)

{'all', 'needn', 'because', 'up', "she'll", 'or', "you've", "shan't", 'over', 'where', 'into', 'out', "i'll", 'that', 'yourself', 'by', 'same', 'at', 'before', 'hasn', 'll', 'mustn', 'when', 'it', 'very', "they'd", 'you', 'after', 'the', "you'd", 'their', 'those', "mightn't", 'which', 'only', 'what', 'i', 'has', 'will', 'can', 'each', 'she', 're', "that'll", "they've", 'this', "hasn't", "he'd", 'down', "needn't", 'isn', 'm', 'her', 'between', 'whom', 'such', 'is', 'just', 'hers', 'haven', "i'm", 'and', 'most', 'own', "they're", "i've", 'there', 'during', 'through', "he's", 'ours', 'doing', 'am', 'mightn', "it'll", 'we', 'how', 'itself', 'y', 'our', 'any', 'once', 'few', 'are', 'd', 'these', 'as', 'ma', 'himself', 'both', "couldn't", 'having', 'me', 'than', "it's", 'o', "you'll", 'be', 'some', "we're", 't', 's', 'had', 'against', "i'd", 'aren', 'should', 'have', 'further', 'with', 'a', 'too', 'them', 'were', 'ourselves', 'theirs', "we'll", 'but', 'off', 'for', 'they', 'themselves', 'was

In [618]:
urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
userPattern = '@[^\s]+'
# 去除标点符号，保留'
punctuationPattern = r"[^\w\s']|(?<!\w)n\'t"
def process_tweets_no_negation_stopword_handling(tweet):
  # Lower Casing
    tweet = tweet.lower()
    # tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    #tokenizing words
    tokens = word_tokenize(tweet)
    #Removing Stop Words
    # final_tokens = [w for w in tokens if w not in stopword]
    final_tokens = tokens
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

def process_tweets_negation_stopword_handling_fix(tweet):
    # Lower Casing
    tweet = tweet.lower()
    #tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    # 移除非字母数字、空格和'的字符
    tweet = re.sub(r"[^\w\s']", '', tweet)
    # --- 特殊否定缩写处理 ---
    # 1. 明确处理 can't → cannot
    tweet = re.sub(r"\bcan't\b", "cannot", tweet)
    # 2. 处理 won't → will not
    tweet = re.sub(r"\bwon't\b", "will not", tweet)
    # 3. 处理 shan't → shall not
    tweet = re.sub(r"\bshan't\b", "shall not", tweet)
    # 4. 处理 ain't → is not
    tweet = re.sub(r"\bain't\b", "is not", tweet)
    # --- 通用n't处理 ---
    # 将剩余n't替换为 not（如didn't → did not）
    tweet = re.sub(r"n't\b", " not", tweet)
    # --- 处理其他撇号为空格（如I'll → I ll）---
    tweet = re.sub(r"'", ' ', tweet)
    print(f"tweet: {tweet}")
    #tokenizing words
    tokens = word_tokenize(tweet)
    print(f"before removing stopword tokens: {tokens}")

    # 移除非否定停用词
    negation_words = {
        # 基础否定词
        "not", "no", "never", "nor", "none", "nobody", "nothing",
        "neither", "nowhere", "without", "cannot"
        # 预处理后可能保留的否定词（如来自特殊缩写替换）
        # ,"isn", "aren", "wasn", "weren"
    }
    custom_stop_words = stopword.difference(negation_words)
    final_tokens = [w for w in tokens if w not in custom_stop_words]
    print(f"after removing stopword tokens: {final_tokens}")

    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]

    for w in final_tokens:
        if len(w) > 1:
            # 动词词性还原（解决 does→doe 问题）
            word = wordLemm.lemmatize(w, pos='v')
            # 名词词性还原（根据任务需求）
            word = wordLemm.lemmatize(word, pos='n')
            finalwords.append(word)
    return ' '.join(finalwords)

def process_tweets_negation_handling_fix(tweet):
    # Lower Casing
    tweet = tweet.lower()
    #tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    # 移除非字母数字、空格和'的字符
    tweet = re.sub(r"[^\w\s']", '', tweet)
    # --- 特殊否定缩写处理 ---
    # 1. 明确处理 can't → cannot
    tweet = re.sub(r"\bcan't\b", "cannot", tweet)
    # 2. 处理 won't → will not
    tweet = re.sub(r"\bwon't\b", "will not", tweet)
    # 3. 处理 shan't → shall not
    tweet = re.sub(r"\bshan't\b", "shall not", tweet)
    # 4. 处理 ain't → is not
    tweet = re.sub(r"\bain't\b", "is not", tweet)
    # --- 通用n't处理 ---
    # 将剩余n't替换为 not（如didn't → did not）
    tweet = re.sub(r"n't\b", " not", tweet)
    # --- 处理其他撇号为空格（如I'll → I ll）---
    tweet = re.sub(r"'", ' ', tweet)
    print(f"tweet: {tweet}")

    #tokenizing words
    tokens = word_tokenize(tweet)
    # Remove punctuations but remain ‘n't’
    final_tokens = tokens
    print(f"tokens: {tokens}")
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]

    for w in tokens:
        if len(w) > 1:
            # 动词词性还原（解决 does→doe 问题）
            word = wordLemm.lemmatize(w, pos='v')
            # 名词词性还原（根据任务需求）
            word = wordLemm.lemmatize(word, pos='n')
            finalwords.append(word)
    return ' '.join(finalwords)

# 数据处理方法有点问题，tweet = tweet.translate(str.maketrans("","",string.punctuation))
# 去除了标点符号，导致带有'n't'的否定词无法被识别为negative word，
# 如何修复既能识别到带有'n't'的否定词，又能去除其它标点符号
# 还需要注意类似I'll要改为I ll
def process_tweets_negation_handling(tweet):
   # Lower Casing
    tweet = tweet.lower()
    #tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    print(f"tweet: {tweet}")
    #tokenizing words
    tokens = word_tokenize(tweet)
    print(f"tokens: {tokens}")
    #Removing Stop Words
    #final_tokens = [w for w in tokens if w not in stopword]
    final_tokens = tokens
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    negation_words = {"not", "no", "never", "n't"}
    prev_neg = False

    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        #finalwords.append(word)
        if word in negation_words:
          prev_neg = True
        elif prev_neg:
          finalwords.append("not_" + word)
          prev_neg = False
        else:
          finalwords.append(word)
    return ' '.join(finalwords)

# punctuations处理会让"didn't" 变成 "didnt"，这不在stopword的否定词（didn,didn't），可以留下
# punctuations处理会让"i'll" 变成 "ill" 和其它单词混淆
def process_tweets_negation_stopword_handling(tweet):
    # Lower Casing
    tweet = tweet.lower()
    #tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    print(f"Remove punctuations: {tweet}")
    #tokenizing words
    tokens = word_tokenize(tweet)
    #Removing Stop Words
    negation_words = {"no", "nor", "not", "don", "don't", "didn", "didn't", "doesn", "doesn't", "can't", "cannot", "won't", "wouldn", "wouldn't", "shouldn", "shouldn't", "isn't", "aren't", "wasn't", "weren't"}

    # Remove negation words from the stop word list
    custom_stop_words = stopword.difference(negation_words)
    # print(f"custom_stop_words: {custom_stop_words}")

    final_tokens = [w for w in tokens if w not in custom_stop_words]
    print(f"tokens: {tokens}")
    #final_tokens = tokens
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

def process_tweets_stopword_handling(tweet):
  # Lower Casing
    tweet = tweet.lower()
    # tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    #tokenizing words
    tokens = word_tokenize(tweet)
    print(f"tokens: {tokens}")
    #Removing Stop Words
    final_tokens = [w for w in tokens if w not in stopword]
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

In [619]:
def tokenizer(max_words, test_text):
    # 1. build word table with top 50 words
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(test_text)
    print("单词表:", tokenizer.word_index)

    sequences = tokenizer.texts_to_sequences(test_text)
    print("文本的单词索引序列:", sequences)

    tweets = pad_sequences(sequences, maxlen=10)
    print(tweets.shape)
    print("填充后文本的单词索引序列:", tweets)

In [620]:
# Test
max_words = 100
test_tweets = [
    "Aww... @Wowena just won't got home from church??? He doesn't go today",
    "You aren't happy with the tables at all",
    "i'll mightn't come back! website at https://www.example.com",
    "They ain't happy with the service at all."
]

processed_test_text = [process_tweets_negation_handling_fix(x) for x in test_tweets]
print("预处理后的数据:", processed_test_text)
tokenizer(max_words, processed_test_text)

tweet: aww  just will not got home from church he does not go today
tokens: ['aww', 'just', 'will', 'not', 'got', 'home', 'from', 'church', 'he', 'does', 'not', 'go', 'today']
tweet: you are not happy with the tables at all
tokens: ['you', 'are', 'not', 'happy', 'with', 'the', 'tables', 'at', 'all']
tweet: i ll might not come back website at 
tokens: ['i', 'll', 'might', 'not', 'come', 'back', 'website', 'at']
tweet: they is not happy with the service at all
tokens: ['they', 'is', 'not', 'happy', 'with', 'the', 'service', 'at', 'all']
预处理后的数据: ['aww just will not get home from church he do not go today', 'you be not happy with the table at all', 'll might not come back website at', 'they be not happy with the service at all']
单词表: {'not': 1, 'at': 2, 'be': 3, 'happy': 4, 'with': 5, 'the': 6, 'all': 7, 'aww': 8, 'just': 9, 'will': 10, 'get': 11, 'home': 12, 'from': 13, 'church': 14, 'he': 15, 'do': 16, 'go': 17, 'today': 18, 'you': 19, 'table': 20, 'll': 21, 'might': 22, 'come': 23, 'ba